In [2]:
import ftplib
import os
import pandas as pd

In [12]:
airData = pd.read_csv("airCSVData/midas_tempdrnl_200001-200012.csv",sep=r'\s*,\s*',
                           header=0, encoding='ascii', engine='python')

In [4]:
soilData = pd.read_csv("soilCSVData/midas_soiltemp_200001-200012.csv",sep=r'\s*,\s*',
                           header=0, encoding='ascii', engine='python')

In [5]:
#print(airData.dtypes)
#print(soilData.dtypes)

OB_END_TIME          object
ID_TYPE              object
ID                  float64
OB_HOUR_COUNT       float64
VERSION_NUM         float64
MET_DOMAIN_NAME      object
SRC_ID              float64
REC_ST_IND          float64
MAX_AIR_TEMP        float64
MIN_AIR_TEMP        float64
MIN_GRSS_TEMP       float64
MIN_CONC_TEMP       float64
MAX_AIR_TEMP_Q      float64
MIN_AIR_TEMP_Q      float64
MIN_GRSS_TEMP_Q     float64
MIN_CONC_TEMP_Q     float64
METO_STMP_TIME       object
MIDAS_STMP_ETIME    float64
MAX_AIR_TEMP_J       object
MIN_AIR_TEMP_J       object
MIN_GRSS_TEMP_J     float64
MIN_CONC_TEMP_J     float64
dtype: object
ID                      int64
ID_TYPE                object
OB_TIME                object
MET_DOMAIN_NAME        object
VERSION_NUM           float64
SRC_ID                float64
REC_ST_IND            float64
Q5CM_SOIL_TEMP        float64
Q10CM_SOIL_TEMP       float64
Q20CM_SOIL_TEMP       float64
Q30CM_SOIL_TEMP       float64
Q50CM_SOIL_TEMP       float64
Q100CM_SOI

In [22]:
airDataS = airData.drop(columns = ['MAX_AIR_TEMP_J','MIN_AIR_TEMP_J','MIN_GRSS_TEMP_J','MIN_CONC_TEMP_J',
                                 'MAX_AIR_TEMP_Q','MIN_AIR_TEMP_Q','MIN_GRSS_TEMP_Q','MIN_CONC_TEMP_Q'])

airDataS = airDataS.drop(columns = ['ID_TYPE','VERSION_NUM','MAX_AIR_TEMP','MIN_AIR_TEMP',
                                 'MET_DOMAIN_NAME','ID','MIDAS_STMP_ETIME','MIN_CONC_TEMP','REC_ST_IND',
                                 'OB_HOUR_COUNT','METO_STMP_TIME'])

#print(airDataS.iloc[1])
airDataS[['OB_END_DATE','OB_END_TIME']] = airDataS['OB_END_TIME'].str.split(' ',expand=True)

#airLocation = (airDataS['SRC_ID'] == 1039)
grassTemp = airDataS.MIN_GRSS_TEMP.notnull()

airFilter = grassTemp #& airLocation

filteredAirSet = airDataS[airFilter]
print(filteredAirSet)

       OB_END_TIME  SRC_ID  MIN_GRSS_TEMP OB_END_DATE
9            09:00   697.0            4.8  2000-01-01
11           09:00   831.0            2.6  2000-01-01
16           09:00     1.0            3.2  2000-01-01
17           09:00     3.0            4.7  2000-01-01
18           09:00     9.0            2.5  2000-01-01
...            ...     ...            ...         ...
346644       09:00  1609.0            6.6  2000-12-31
346645       09:00  1572.0           -5.7  2000-12-31
346647       09:00  1575.0           -5.1  2000-12-31
346648       09:00  1577.0           -6.1  2000-12-31
346649       09:00  1579.0           -6.7  2000-12-31

[199030 rows x 4 columns]
550


In [14]:
cmTest = (soilData['Q100CM_SOIL_TEMP'] > 0)
locationTest = (soilData['SRC_ID'] == 1039)

newestTest = cmTest #& locationTest

filteredSet = soilData[newestTest]

filteredSet = filteredSet.drop(columns = ['Q100CM_SOIL_TEMP_J','Q50CM_SOIL_TEMP_J','Q30CM_SOIL_TEMP_J','Q20CM_SOIL_TEMP_J',
                                        'Q10CM_SOIL_TEMP_J','Q100CM_SOIL_TEMP_Q','Q50CM_SOIL_TEMP_Q','Q30CM_SOIL_TEMP_Q',
                                        'Q20CM_SOIL_TEMP_Q','Q10CM_SOIL_TEMP_Q','Q5CM_SOIL_TEMP_Q','Q5CM_SOIL_TEMP_J',
                                        'Q50CM_SOIL_TEMP','Q30CM_SOIL_TEMP','Q20CM_SOIL_TEMP',
                                        'Q10CM_SOIL_TEMP','Q5CM_SOIL_TEMP', 'ID', 'ID_TYPE', 'MET_DOMAIN_NAME', 
                                        'VERSION_NUM', 'MIDAS_STMP_ETIME','METO_STMP_TIME','REC_ST_IND'])

filteredSet[['OB_DATE','OB_TIME']] = filteredSet['OB_TIME'].str.split(' ',expand=True)

print(filteredSet)#.hist('Q100CM_SOIL_TEMP', bins = 'auto'))

       OB_TIME   SRC_ID  Q100CM_SOIL_TEMP     OB_DATE
740      09:00      3.0               6.0  2000-01-01
741      09:00      9.0               5.6  2000-01-01
744      09:00     23.0               5.0  2000-01-01
747      09:00     44.0               5.3  2000-01-01
749      09:00     52.0               5.7  2000-01-01
...        ...      ...               ...         ...
943951   09:00  30103.0               4.4  2000-12-31
943952   09:00   1580.0               6.8  2000-12-31
943953   09:00   1582.0               4.9  2000-12-31
943955   09:00   1572.0               8.4  2000-12-31
943956   09:00   1575.0               8.5  2000-12-31

[67853 rows x 4 columns]
201
195


In [23]:
noOfStationsGrass = filteredAirSet.SRC_ID.unique().tolist()
noOfStationsSoil = filteredSet.SRC_ID.unique().tolist()

bothHaveData = list(set(noOfStationsSoil) & set(noOfStationsGrass))

print(len(bothValid))

195


In [8]:
##STACK OVE

from datetime import timedelta, date

def daterange(start_date, end_date):
    for n in range(int((end_date - start_date).days)):
        yield start_date + timedelta(n)

start_date = date(2013, 1, 1)
end_date = date(2015, 6, 2)
for single_date in daterange(start_date, end_date):
    print(single_date.strftime("%Y-%m-%d"))

2013-01-01
2013-01-02
2013-01-03
2013-01-04
2013-01-05
2013-01-06
2013-01-07
2013-01-08
2013-01-09
2013-01-10
2013-01-11
2013-01-12
2013-01-13
2013-01-14
2013-01-15
2013-01-16
2013-01-17
2013-01-18
2013-01-19
2013-01-20
2013-01-21
2013-01-22
2013-01-23
2013-01-24
2013-01-25
2013-01-26
2013-01-27
2013-01-28
2013-01-29
2013-01-30
2013-01-31
2013-02-01
2013-02-02
2013-02-03
2013-02-04
2013-02-05
2013-02-06
2013-02-07
2013-02-08
2013-02-09
2013-02-10
2013-02-11
2013-02-12
2013-02-13
2013-02-14
2013-02-15
2013-02-16
2013-02-17
2013-02-18
2013-02-19
2013-02-20
2013-02-21
2013-02-22
2013-02-23
2013-02-24
2013-02-25
2013-02-26
2013-02-27
2013-02-28
2013-03-01
2013-03-02
2013-03-03
2013-03-04
2013-03-05
2013-03-06
2013-03-07
2013-03-08
2013-03-09
2013-03-10
2013-03-11
2013-03-12
2013-03-13
2013-03-14
2013-03-15
2013-03-16
2013-03-17
2013-03-18
2013-03-19
2013-03-20
2013-03-21
2013-03-22
2013-03-23
2013-03-24
2013-03-25
2013-03-26
2013-03-27
2013-03-28
2013-03-29
2013-03-30
2013-03-31
2013-04-01